In [1]:
import socket,time
import struct
def spliter(format,buffer):
    data=buffer[:struct.calcsize(format)]
    buffer=buffer[struct.calcsize(format):]
    ret=struct.unpack(format,data)
    if len(ret)==1:ret=ret[0]
    return ret,buffer

from collections import deque
class ESP:
    """make udp connections"""
    def __init__(self,place,self_port=9999,remote_port=8080,max_rate=1000,low_rate=100,save=10):
        if type(place)==int:
            place=str(place)
            place="192.168.1." + place
        self.place=place
        self.self_port=self_port
        self.remote_port = remote_port
        self.self_port=self_port
        self.udp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.udp.setblocking(False)
        self.max_rate=max_rate
        self.low_rate=low_rate

        self.connect()

        self.encode_list = []
        self.decode_list = []
        self.encode_list.append(["I", "get_num"])

        self.decode_list.append(["I", "to_num"])
        self.decode_list.append(["b", "get_rssi"])

        self.mode = 0

        # self.save=save
        self.save_num=0

    last_value=b""
    def send(self,buf):
        num,buffer=spliter("I",buf)
        if self.last_value!=buffer:
            self.save_num=num
            self.send_(buf)
        else:
            if not self.save_num<self.to_num:
                self.send_(buf)
            elif self.get_interval() > 60 / self.low_rate:self.send_(buf)

        self.last_value=buffer

    def send_(self,buffer):
        self.udp.send(buffer)
        self.last_get = time.perf_counter()

    def connect(self):
        n = socket.gethostname()
        bind=socket.gethostbyname_ex(n)[-1][-1]
        self.udp.bind((bind,self.self_port))
        self.udp.connect((self.place,self.remote_port))
        
        
    last_get=None
    def get_interval(self):
        if self.last_get is None:
            self.last_get=time.perf_counter()
            return
        ret=time.perf_counter()-self.last_get
        return ret

    num = 0
    def get_num(self):
        self.num += 1
        return self.num
    
    last=0
    def last_recv(self,check=True,wait=5):
        #检查是否是同一个连接
        if check:
            return self.get_time() - self.last < wait
        else:self.last=self.get_time()
    
    def get_time(self):return time.perf_counter()

    to_num=0
    def decode(self, buffer):
        self.last_recv(0)
#         print(buffer)
        for form, name in self.decode_list:
            data, buffer = spliter(form, buffer)

            if name=="to_num":
                if self.to_num>data and self.last_recv():break
            
            if (not name in self.__dict__) or not callable(self.__dict__[name]):
                self.__dict__[name] = data
            elif hasattr(self, name):
                if type(data)==tuple:
                    getattr(self, name)(*data)
                else:getattr(self, name)(data)

    def encode(self):
        buffer = b""
        for form, name in self.encode_list:
            if name in self.__dict__ and not callable(self.__dict__[name]):
                data = self.__dict__[name]
            elif hasattr(self,name):
                data = getattr(self,name)()
            if type(data) in [tuple,list]:buffer += struct.pack(form, *data)
            else:buffer += struct.pack(form, data)
        return buffer

    def __call__(self, *args, **kwargs):
        g=self.get_interval()
        if g is None or g>60/self.max_rate:
            self.send(self.encode())
        if True:
            try:
                recv = self.udp.recv(8192)
                self.decode(recv)
                return True
            except Exception as e:
                if not e.args[0] == 11:
                    print(e)
                    raise e


class Controller(ESP):
    def add_control_set(self, lis):
        self.encode_list.append(lis)
    def add_recv_set(self,lis):
        self.decode_list.append(lis)

    def 延迟(self):
        self.num

    def 丢包率(self):
        self.num

In [2]:
bind=socket.gethostbyname_ex(socket.gethostname())[-1][-1]
bind

'192.168.160.2'

In [3]:
esp=Controller("192.168.2.123",self_port=5008)

In [4]:
esp.place,esp.remote_port

('192.168.2.123', 8080)

In [5]:
esp()

In [6]:
esp.udp

<socket.socket fd=74, family=AddressFamily.AF_INET, type=SocketKind.SOCK_DGRAM, proto=0, laddr=('192.168.160.2', 5008), raddr=('192.168.2.123', 8080)>

In [7]:
esp.add_recv_set(["6I", "C"])

In [8]:
%run from_empyty_list.ipynb

In [9]:
esp.C=C
C.li,C.now_num

([], 0)

In [62]:
for i in range(1000):esp()
C.li

[205,
 206,
 207,
 208,
 208,
 210,
 211,
 213,
 218,
 218,
 226,
 242,
 283,
 370,
 226,
 242,
 283,
 370,
 533]

In [63]:
len(C.li)

19